In [1]:
import os
import numpy as np
import cv2
from os import listdir, mkdir
from os.path import isfile, join, isdir
import dlib
from PIL import Image
from numba import njit



In [5]:
# i = 1
# dirs =  os.listdir('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/test')
# dirs.sort()
# for dir in dirs:
#     for name in os.listdir(os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/test',dir)):
#         src_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/test',dir,name)
#         des_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/test',dir,str(i).zfill(6)+'.jpg')
#         os.rename(src_path,des_path)
#         i+=1

# i = 1
# dirs =  os.listdir('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/train')
# dirs.sort()
# for dir in dirs:
#     for name in os.listdir(os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/train',dir)):
#         src_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/train',dir,name)
#         des_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/train',dir,str(i).zfill(6)+'.jpg')
#         os.rename(src_path,des_path)
#         i+=1

# dirs =  os.listdir('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_train/clr')
# dirs.sort()
# for dir in dirs:
#     for name in os.listdir(os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_train/clr',dir)):
#         src_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_train/clr',dir,name)
#         if 'feature_refined' in name:
#             os.remove(src_path)
dirs =  os.listdir('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_test/clr')
dirs.sort()
for dir in dirs:
    for name in os.listdir(os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_test/clr',dir)):
        src_path = os.path.join('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_test/clr',dir,name)
        if 'feature_refined' in name:
            os.remove(src_path)



In [12]:
def get_lndm(path_img, path_out, start_id = 0, dlib_path=""):
    dir_proc = {'msk':'msk', 'org':'orig', 'clr':'clr', 'lnd':'lndm'}

    for dir_it in dir_proc:
        if os.path.isdir(path_out + dir_proc[dir_it]) == False:
            os.mkdir(path_out + dir_proc[dir_it])

    folder_list = [f for f in listdir(path_img)]
    folder_list.sort()

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(dlib_path+"shape_predictor_68_face_landmarks.dat")

    line_px = 1
    res_w = 256
    res_h = 256

    for fld in folder_list[:]:
        imglist_all = [f[:-4] for f in listdir(join(path_img, fld)) if isfile(join(path_img, fld, f)) and f[-4:] == ".jpg"]
        imglist_all.sort(key=int)
        imglist_all = imglist_all[start_id:]

        for dir_it in dir_proc:
            if os.path.isdir(join(path_out, dir_proc[dir_it], fld)) == False:
                os.mkdir(join(path_out, dir_proc[dir_it], fld))

        land_mask = True
        crop_coord = []
        for it in range(len(imglist_all)):
            clr = cv2.imread(join(path_img, fld, imglist_all[it]+".jpg"), cv2.IMREAD_ANYCOLOR)
            img = clr.copy()
            img_dlib = np.array(clr[:, :, :], dtype=np.uint8)
            dets = detector(img_dlib, 1)

            for k_it, d in enumerate(dets):
                if k_it != 0:
                    continue
                landmarks = predictor(img_dlib, d)

                # centering
                c_x = int((landmarks.part(42).x + landmarks.part(39).x) / 2)
                c_y = int((landmarks.part(42).y + landmarks.part(39).y) / 2)
                w_r = int((landmarks.part(42).x - landmarks.part(39).x)*4)
                h_r = int((landmarks.part(42).x - landmarks.part(39).x)*5)
                w_r = int(h_r/res_h*res_w)

                w, h = int(w_r * 2), int(h_r * 2)
                pd = int(w) # padding size
                
                img_p = np.zeros((img.shape[0]+pd*2, img.shape[1]+pd*2, 3), np.uint8) * 255
                img_p[:, :, 0] = np.pad(img[:, :, 0], pd, 'edge')
                img_p[:, :, 1] = np.pad(img[:, :, 1], pd, 'edge')
                img_p[:, :, 2] = np.pad(img[:, :, 2], pd, 'edge')
                
                visual = img_p[c_y - h_r+pd:c_y + h_r+pd, c_x - w_r+pd:c_x + w_r+pd]

                crop_coord.append([c_y - h_r, c_y + h_r, c_x - w_r, c_x + w_r, pd, imglist_all[it]+".jpg"])
                t_x, t_y = int(c_x - w_r), int(c_y - h_r)

                ratio_w, ratio_h = res_w/w, res_h/h

                visual = cv2.resize(visual, dsize=(res_w, res_h), interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(join(path_out, dir_proc['clr'], fld, imglist_all[it]+".jpg"), visual) #saving crop
                cv2.imwrite(join(path_out, dir_proc['org'], fld, imglist_all[it]+".jpg"), clr) # saving original

                 

        #np.save(join(path_out, dir_proc['org'], fld, 'crop_coord.npy'), crop_coord) #crop coordinates
        print("folder done",fld)



# get_lndm('/home/junjie/DeIDVideo/CelebA/clr_test', 
#               '/home/junjie/DeIDVideo/SemanticImageSynthesis/ciagan_segmentation/CeleBAT/',
#              dlib_path='/home/junjie/DeIDVideo/ciagan/source/')
get_lndm('/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience/test',
    '/home/qiuyang/anonymous/ciagan_semantic/dataset/Adience_test/',
    dlib_path='/home/junjie/DeIDVideo/ciagan/source/')   

folder done 0
folder done 1
folder done 10
folder done 11
folder done 12
folder done 13
folder done 14
folder done 15
folder done 16
folder done 17
folder done 18
folder done 19
folder done 2
folder done 20
folder done 21
folder done 22
folder done 23
folder done 24
folder done 25
folder done 26
folder done 27
folder done 28
folder done 29
folder done 3
folder done 30
folder done 31
folder done 32
folder done 33
folder done 34
folder done 35
folder done 36
folder done 37
folder done 38
folder done 39
folder done 4
folder done 40
folder done 41
folder done 42
folder done 43
folder done 44
folder done 45
folder done 46
folder done 47
folder done 48
folder done 49
folder done 5
folder done 50
folder done 51
folder done 52
folder done 53
folder done 54
folder done 55
folder done 56
folder done 57
folder done 58
folder done 59
folder done 6
folder done 60
folder done 61
folder done 62
folder done 63
folder done 64
folder done 65
folder done 66
folder done 67
folder done 68
folder done 69
fo